In [98]:
import gspread
import fairpy

account = gspread.service_account("credentials.json")
spreadsheet = account.open("FairDivision")
input = spreadsheet.worksheet("input")
print("Rows: ", input.row_count, "Cols: ", input.col_count)
rows = input.get_all_values()
print(rows)

Rows:  1000 Cols:  27
[['מפלגה  v  ', 'מנדטים v   משרד >', 'חוץ', 'בטחון', 'אוצר', 'בט"פ', 'משפטים', 'פנים', 'בריאות', '', 'סה"כ'], ['הליכוד', '32', '20', '20', '20', '10', '10', '10', '10', '', '100'], ['הציונות הדתית', '14', '10', '20', '10', '30', '20', '10', '20', '', '120'], ['ש"ס', '11', '5', '5', '20', '5', '10', '30', '20', '', '95'], ['אגודת ישראל', '7', '5', '5', '5', '5', '5', '10', '20', '', '55'], ['סה"כ', '64', '', '', '', '', '', '', '', '', '']]


In [99]:

try:
	output = spreadsheet.worksheet("output")
except gspread.WorksheetNotFound:
	output = spreadsheet.add_worksheet(title="output", rows=len(agents)+2, cols=len(items)+3)
	# TODO: change worksheet direction to RTL
	# I did not find here https://docs.gspread.org/en/latest/api/models/worksheet.html#id1   how to do this.
input_range = input.range(1, 1, len(rows), len(rows[0]))
output.update_cells(input_range)


{'spreadsheetId': '1tJPV-y-r1TAx5FqbrqecKPJMeKHTtIDeiYck8eLoGKY',
 'updatedRange': 'output!A1:K6',
 'updatedRows': 6,
 'updatedColumns': 11,
 'updatedCells': 66}

## Read input

In [106]:
items = rows[0][2:-1]  # remove agent names, entitlements, and total
items.remove('')
print("items: ", items)

items:  ['חוץ', 'בטחון', 'אוצר', 'בט"פ', 'משפטים', 'פנים', 'בריאות']


In [107]:
rows_of_agents = rows[1:-1]    # remove item names and total
agents = [row[0] for row in rows_of_agents]         
print("agents: ", agents)

agents:  ['הליכוד', 'הציונות הדתית', 'ש"ס', 'אגודת ישראל']


In [108]:
entitlements = [row[1] for row in rows_of_agents]   
print("entitlements: ", entitlements)

entitlements:  ['32', '14', '11', '7']


In [111]:
def row_to_prefs(row:list)->list:
	prefs_list = row[2:-1]   # Remove party name, party entitlement, and total
	prefs_dict = {}
	for o in range(len(items)):
		value = prefs_list[o]
		value = 0.0 if value=='' else float(value)
		prefs_dict[items[o]] = value
	return prefs_dict
preferences = {row[0]: row_to_prefs(row) for row in rows_of_agents}
print("preferences: ",preferences)

preferences:  {'הליכוד': {'חוץ': 20.0, 'בטחון': 20.0, 'אוצר': 20.0, 'בט"פ': 10.0, 'משפטים': 10.0, 'פנים': 10.0, 'בריאות': 10.0}, 'הציונות הדתית': {'חוץ': 10.0, 'בטחון': 20.0, 'אוצר': 10.0, 'בט"פ': 30.0, 'משפטים': 20.0, 'פנים': 10.0, 'בריאות': 20.0}, 'ש"ס': {'חוץ': 5.0, 'בטחון': 5.0, 'אוצר': 20.0, 'בט"פ': 5.0, 'משפטים': 10.0, 'פנים': 30.0, 'בריאות': 20.0}, 'אגודת ישראל': {'חוץ': 5.0, 'בטחון': 5.0, 'אוצר': 5.0, 'בט"פ': 5.0, 'משפטים': 5.0, 'פנים': 10.0, 'בריאות': 20.0}}


In [112]:
preferences[agents[0]]

{'חוץ': 20.0,
 'בטחון': 20.0,
 'אוצר': 20.0,
 'בט"פ': 10.0,
 'משפטים': 10.0,
 'פנים': 10.0,
 'בריאות': 10.0}

In [113]:
allocation = fairpy.items.leximin_optimal_allocation(preferences)
allocation

הליכוד gets { 61.364% of חוץ, 100.0% of בטחון} with value 32.3.
הציונות הדתית gets { 100.0% of בט"פ, 11.364% of משפטים} with value 32.3.
ש"ס gets { 100.0% of אוצר, 40.909% of פנים} with value 32.3.
אגודת ישראל gets { 38.636% of חוץ, 88.636% of משפטים, 59.091% of פנים, 100.0% of בריאות} with value 32.3.

In [114]:
map_agent_to_fractions = {agents[i]: allocation.bundles[i].fractions for i in range(len(agents))}
map_agent_to_fractions

{'הליכוד': array([ 0.61363636,  1.        , -0.        , -0.        , -0.        ,
        -0.        , -0.        ]),
 'הציונות הדתית': array([-0.        , -0.        , -0.        ,  1.        ,  0.11363636,
        -0.        , -0.        ]),
 'ש"ס': array([-0.        , -0.        ,  1.        , -0.        , -0.        ,
         0.40909091, -0.        ]),
 'אגודת ישראל': array([ 0.38636364, -0.        , -0.        , -0.        ,  0.88636364,
         0.59090909,  1.        ])}

In [115]:
utility_profile = allocation.utility_profile()
map_agent_to_utility = dict(zip(agents, utility_profile))
map_agent_to_utility

{'הליכוד': 32.27272727272727,
 'הציונות הדתית': 32.27272727272727,
 'ש"ס': 32.27272727272727,
 'אגודת ישראל': 32.272727272727266}

## Create output

In [118]:
for i in range(len(agents)):
	bundle_i = map_agent_to_fractions[agents[i]]
	print(agents[i], ": ", bundle_i)
	for o in range(len(items)):
		fraction_i_o = map_agent_to_fractions[agents[i]][o]
		# print(fraction_i_o)
		output.update_cell(i+2, o+3, fraction_i_o)

[ 0.61363636  1.         -0.         -0.         -0.         -0.
 -0.        ]
[-0.         -0.         -0.          1.          0.11363636 -0.
 -0.        ]
[-0.         -0.          1.         -0.         -0.          0.40909091
 -0.        ]
[ 0.38636364 -0.         -0.         -0.          0.88636364  0.59090909
  1.        ]


In [119]:
utility_column = len(items)+4
output.update_cell(1, utility_column, "תועלת")

{'spreadsheetId': '1tJPV-y-r1TAx5FqbrqecKPJMeKHTtIDeiYck8eLoGKY',
 'updatedRange': 'output!K1',
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}

In [126]:
for i in range(len(agents)):
	# output.update_cell(i+2, utility_column, utility_profile[i])
	row_num = i+2
	first_cell = gspread.utils.rowcol_to_a1(row_num, 3)
	last_cell = gspread.utils.rowcol_to_a1(row_num, len(items)+2)
	range_a1 = f"{first_cell}:{last_cell}"
	output.update_cell(row_num, utility_column, f"=SUMPRODUCT(input!{range_a1},output!{range_a1})")

In [122]:
gspread.utils.rowcol_to_a1(3, 4)

'D3'